# 
## Group 46
### Tsun Li Nicholas Tam, Camila Dall Bello, Leon Zhang


In [ ]:
# Imports
library(cowplot)
library(dplyr)
library(gridExtra)
library(tidyverse)
library(repr)
library(infer)
library(repr)
library(datateachr)
library(digest)
library(lubridate)
library(taxyvr)

## Introduction



## Preliminary Results


## Methods: Plan


## References


In [12]:
# url <- ""
pollution <- read.csv("https://raw.githubusercontent.com/Nick-2003/STAT-201-Group-46-Project/main/en_climate_monthly_BC_1165793_1987-2007_P1M.csv")
pollution

# getwd()

Longitude..x.,Latitude..y.,Station.Name,Climate.ID,Date.Time,Year,Month,Mean.Max.Temp...C.,Mean.Max.Temp.Flag,Mean.Min.Temp...C.,⋯,Total.Snow..cm.,Total.Snow.Flag,Total.Precip..mm.,Total.Precip.Flag,Snow.Grnd.Last.Day..cm.,Snow.Grnd.Last.Day.Flag,Dir.of.Max.Gust..10.s.deg.,Dir.of.Max.Gust.Flag,Spd.of.Max.Gust..km.h.,Spd.of.Max.Gust.Flag
<dbl>,<dbl>,<chr>,<int>,<chr>,<int>,<int>,<dbl>,<chr>,<dbl>,⋯,<dbl>,<chr>,<dbl>,<chr>,<int>,<chr>,<lgl>,<lgl>,<lgl>,<lgl>
-121.22,51.68,100 MILE HOUSE 6NE,1165793,1987-01,1987,1,NA,M,NA,⋯,NA,M,NA,M,NA,M,NA,NA,NA,NA
-121.22,51.68,100 MILE HOUSE 6NE,1165793,1987-02,1987,2,NA,M,NA,⋯,NA,M,NA,M,NA,M,NA,NA,NA,NA
-121.22,51.68,100 MILE HOUSE 6NE,1165793,1987-03,1987,3,NA,M,NA,⋯,NA,M,NA,M,NA,M,NA,NA,NA,NA
-121.22,51.68,100 MILE HOUSE 6NE,1165793,1987-04,1987,4,NA,M,NA,⋯,NA,M,NA,M,NA,M,NA,NA,NA,NA
-121.22,51.68,100 MILE HOUSE 6NE,1165793,1987-05,1987,5,NA,M,NA,⋯,NA,M,NA,M,NA,M,NA,NA,NA,NA
-121.22,51.68,100 MILE HOUSE 6NE,1165793,1987-06,1987,6,NA,M,NA,⋯,0.0,,NA,M,0,,NA,NA,NA,NA
-121.22,51.68,100 MILE HOUSE 6NE,1165793,1987-07,1987,7,23.2,,6.9,⋯,0.0,,61.6,,0,,NA,NA,NA,NA
-121.22,51.68,100 MILE HOUSE 6NE,1165793,1987-08,1987,8,21.5,,3.8,⋯,0.0,,23.0,,0,,NA,NA,NA,NA
-121.22,51.68,100 MILE HOUSE 6NE,1165793,1987-09,1987,9,21.8,,0.7,⋯,0.0,,2.2,,0,,NA,NA,NA,NA
